# Worldclim pre-process to arctic

The originally downloaded worldclim files are global in extent, so here we pre-process to reduce the data to just the area we are interested in, the arctic in this case.

This step will open an extent `.tif` and a bunch of Worldclim `.zip` files. It will crop to the extents of the `.tif` (plus a buffer; default to 30px) and reproject. The result will be in memory and then you have to save it out to an explicit location. Here it makes sense to save the outputs with a numeric indicator in the file name so that we can tell what processing step we are at in the pipeline...

For example the input to this step is from the raw download data directory and the output is saved labeled with 02 in the file name.

In [16]:
from pathlib import Path

from temds.datasources import worldclim

In [ ]:
out_srs = 'EPSG:6931'
extent_file = 'working/aoi_5km_buffer_6931.tiff' # Need to re-arrange the aoi stuff so it should end up in a folder labeled 01

In [ ]:
# config = 'my-config.yml'
# wc_test = subprograms.download('worldclim', config=config)

# Takes about 20 minutues on a 8 core 32GBmemory machachine.

# This will create an object that has all the data in memory that is cropped
# and re-projected to the extent of the AOI.
wc = worldclim.WorldClim(
    'working/download/worldclim', # This is processing step "1": raw download.
    extent_file,
    verbose = True,
    cleanup_uncompressed = False
)


extent (-4669260.0, 4192980.0, 4366000.0, -3485000.0)
resolution, 4000.0,-4000.0
out size 2242, 1934
Extracting working/download/worldclim/wc2.1_30s_tavg.zip
Extracting where='working/download/worldclim/tavg'
setting tavg from raw data at 'working/download/worldclim'
loading tavg data from working/download/worldclim/tavg/wc2.1_30s_tavg_01.tif at index 0
.. Running gdal.Warp to extent (-4669260.0, -3481000.0, 4366740.0, 4199000.0)
loading tavg data from working/download/worldclim/tavg/wc2.1_30s_tavg_02.tif at index 1
.. Running gdal.Warp to extent (-4669260.0, -3481000.0, 4366740.0, 4199000.0)
loading tavg data from working/download/worldclim/tavg/wc2.1_30s_tavg_03.tif at index 2
.. Running gdal.Warp to extent (-4669260.0, -3481000.0, 4366740.0, 4199000.0)
loading tavg data from working/download/worldclim/tavg/wc2.1_30s_tavg_04.tif at index 3
.. Running gdal.Warp to extent (-4669260.0, -3481000.0, 4366740.0, 4199000.0)
loading tavg data from working/download/worldclim/tavg/wc2.1_30s_tav

In [13]:
wc.dataset

<xarray.Dataset> Size: 1GB
Dimensions:      (time: 12, lat: 1934, lon: 2242)
Coordinates:
  * lat          (lat) float64 15kB -3.483e+06 -3.479e+06 ... 4.249e+06
  * lon          (lon) float64 18kB -4.6e+06 -4.596e+06 ... 4.36e+06 4.364e+06
  * time         (time) int64 96B 1 32 60 91 121 152 182 213 244 274 305 335
    spatial_ref  int64 8B 0
Data variables:
    tavg         (time, lat, lon) float32 208MB nan nan nan nan ... nan nan nan
    tmax         (time, lat, lon) float32 208MB nan nan nan nan ... nan nan nan
    tmin         (time, lat, lon) float32 208MB nan nan nan nan ... nan nan nan
    prec         (time, lat, lon) float32 208MB nan nan nan nan ... nan nan nan
    srad         (time, lat, lon) float32 208MB nan nan nan nan ... nan nan nan
    wind         (time, lat, lon) float32 208MB nan nan nan nan ... nan nan nan
    vapr         (time, lat, lon) float32 208MB nan nan nan nan ... nan nan nan

In [ ]:
# Think we fixed this bug, see datasources/base.py
!mkdir working/02-arctic/worldclim

In [ ]:
wc.save('working/02-arctic/worldclim/worldclim-arctic-buffered-better.nc', overwrite=True)

